# ETL
## Trabalho Final | Prática | Grupo G
### Alunos
- Victor
- Lucas
- Yudi

## Setup

In [41]:
pip install opendatasets pandas pandasql findspark pyspark --quiet

Note: you may need to restart the kernel to use updated packages.


In [2]:
from pandasql import sqldf
import opendatasets as od
import pandas as pds
import os
import re

In [3]:
import findspark
findspark.init()
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkContext('local')
spark = SparkSession(sc)

23/01/28 11:18:42 WARN Utils: Your hostname, buendia-Nitro-AN515-51 resolves to a loopback address: 127.0.1.1; using 192.168.1.21 instead (on interface wlp2s0)
23/01/28 11:18:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/28 11:18:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [27]:
from pyspark.sql import functions as F

## Extract

In [6]:
if not os.path.exists('./popular-movies-datasets-58000-movies/'):
    kaggleAPI = input('[TOKEN API] Insire seu Token API do Kaggle:')

    fp = open('kaggle.json', 'w')
    fp.write(kaggleAPI)
    fp.close()

    od.download(
        'https://www.kaggle.com/datasets/whenamancodes/popular-movies-datasets-58000-movies?select=tags.csv')

    os.remove('kaggle.json')

In [7]:
csvs = {}
for dirname, _, filenames in os.walk('./popular-movies-datasets-58000-movies/'):
    for filename in filenames:
        csvs[filename] = os.path.join(dirname, filename)
        print(os.path.join(dirname, filename))

./popular-movies-datasets-58000-movies/genome-scores.csv
./popular-movies-datasets-58000-movies/genome-tags.csv
./popular-movies-datasets-58000-movies/tags.csv
./popular-movies-datasets-58000-movies/ratings.csv
./popular-movies-datasets-58000-movies/links.csv
./popular-movies-datasets-58000-movies/movies.csv


In [14]:
dfs = {}

for filename in csvs:
    file=(csvs[filename])
    newData = spark.read.format("csv").option("header","true").load(file)
    dfs[re.sub('-', '_', re.findall("(.+).csv", filename)[0])] = newData
    newData.createOrReplaceTempView(re.sub('-', '_', re.findall("(.+).csv", filename)[0]))

In [40]:
for df in dfs:
    print(df,'\n |-->', dfs[df], '\n')

genome_scores 
 |--> DataFrame[movieId: string, tagId: string, relevance: string] 

genome_tags 
 |--> DataFrame[tagId: string, tag: string] 

tags 
 |--> DataFrame[userId: string, movieId: string, tag: string, timestamp: string] 

ratings 
 |--> DataFrame[userId: string, movieId: string, rating: string, timestamp: string] 

links 
 |--> DataFrame[movieId: string, imdbId: string, tmdbId: string] 

movies 
 |--> DataFrame[movieId: string, title: string, genres: string] 

